<a href="https://colab.research.google.com/github/bo-bits/nn-zero-to-hero/blob/master/exercises/makemore_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [6]:
# Trigram Model
# W = [(26*26)+1 x (26*26)+1] = [677, 677]

words = open('names.txt', 'r').read().splitlines()



In [46]:
import torch

# Creates a sorted list of all characters in words
chars = sorted(list(set(''.join(words))))

# builds a dictionary where the key (s) is each character from chars,
# and the value (i+1) is the index of that character plus 1.
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
# reverse dictionary to go from index to s
itos = {i:s for s,i in stoi.items()}

# Create a dictionary where each key (b) is a unique bigram
# The dictionary must include bigrams with '.'
null = chars.insert(0,'.')
btoi = {char1 + char2: i for i, (char1, char2) in enumerate(((a, b) for a in chars for b in chars), 0)}

# # My implementation of the same dictionary (which works, but not the best)
# btoi = {}
# i = 1
# for char1 in chars:
#   for char2 in chars:
#     bgrm = char1 + char2
#     btoi[bgrm] = i
#     i+=1

# create the dataset
# Now xs is a tensor with all of the character indices
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = btoi[ch1+ch2]
    ix3 = stoi[ch3]
    xs.append(ix1)
    ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)

number of examples:  17129


In [49]:
27**2

729

E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?


E06: meta-exercise! Think of a fun/interesting exercise and complete it.